In [ ]:
# load dataset
from p_efficientnet_b7_500_new import define
from torchvision.datasets import ImageFolder
from torch.utils.data.dataloader import DataLoader

transforms = define.transform()

test_data = ImageFolder(f"{define.get_datafolder()}/test", transform=transforms)
test_loader = DataLoader(test_data, batch_size=8, shuffle=True)

In [ ]:
# define model
model = define.create_model()
model_desc = define.model_desc()
model_directory_name = define.__file__.replace("\\", "/").split("/")[-2]

In [ ]:
# create runner
import util

runner = util.Runner(model, default_dir=model_directory_name)
print(f"runner use {runner.device}")
runner.load("p_efficientnet_b7_1000/m_0726_17154578038021.pth") # load latest model

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import torch
from torch import nn

targets = []
results = []
def func(data, target, result):
    global targets, results
    targets.append(target)
    results.append(result)
criterion = nn.CrossEntropyLoss() 
runner.test(test_loader, criterion, func)
targets = torch.cat(targets, dim=0)
results = torch.cat(results, dim=0)

conf_matrix = confusion_matrix(results.to('cpu'), targets.to('cpu'))

plt.figure(figsize=(10,10))
sns.heatmap(conf_matrix, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Atual')
plt.show()

In [ ]:
import util

def func(data, target, result):
    data = data.to('cpu')
    corr = [ test_data.classes[x] for x in target ]
    pred = [ test_data.classes[x] for x in result ]
    title = f"corr: {' * '.join(corr)}\npred: {' * '.join(pred)}"
    util.show2(data, title)
runner.test(test_loader, criterion, func)